# Semantics Construction in GF

With GLIF, we usually use MMT to describe the target representation and semantics construction.
However, there are ways to perform the semantics construction directly in GF.
This notebook explores two such ways:
1. Introduce a concrete syntax for the formal representation
2. Create another abstract syntax for the target representation

In [5]:
archive tmpGLIF/examples misc/SemConstrInGF

GF shell will be reloaded Successfully changed archive

## The Grammar

In [6]:
abstract Grammar = {
    cat
        S;
        Agent;
        Verb;
    data
        and_Verb : Verb -> Verb -> Verb;
        and_S : S -> S -> S;
        act : Agent -> Verb -> S;
        ahmed : Agent;
        berta : Agent;
        paint : Verb;
        be_quiet : Verb;
}

Successfully imported Grammar.gf

In [8]:
concrete GrammarEng of Grammar = {
    lincat
        S = Str;
        Agent = Str;
        Verb = Str;
    lin
        and_Verb v1 v2 = v1 ++ "and" ++ v2;
        and_S s1 s2 = s1 ++ "and" ++ s2;
        act a v = a ++ v;
        ahmed = "Ahmed";
        berta = "Berta";
        paint = "paints";
        be_quiet = "is quiet";
}

Successfully imported GrammarEng.gf

In [10]:
-- we can now parse a few simple example sentences:
parse "Ahmed paints and Berta is quiet"

and_S (act ahmed paint) (act berta be_quiet)

In [11]:
parse "Ahmed is quiet and paints"

act ahmed (and_Verb be_quiet paint)

## Approach 1: Another concrete syntax
Here we treat the semantic representation like any other language and translate to it by creating another concrete syntax – as we usually do for translation with GF.

In [16]:
concrete GrammarLog of Grammar = {
    lincat
        S = Str;
        Agent = Str;
        Verb = Str;
    lin
        and_Verb v1 v2 = "(" ++ "λx." ++ v1 ++ "(x)" ++ "∧" ++ v2 ++ "(x)" ++ ")";
        and_S s1 s2 = s1 ++ "∧" ++ s2;
        act a v = v ++ "(" ++ a ++ ")";
        ahmed = "a";
        berta = "b";
        paint = "p";
        be_quiet = "q";
}

Successfully imported GrammarLog.gf

In [17]:
parse -lang=Eng "Ahmed paints and Berta is quiet" | linearize -lang=Log

p ( a ) ∧ q ( b )

In [18]:
parse -lang=Eng "Ahmed is quiet and paints" | linearize -lang=Log

( λx. q (x) ∧ p (x) ) ( a )

Note that *( λx. q (x) ∧ p (x) ) ( a )* β-reduces to *q(a) ∧ p(a)*

## Approach 2: Another abstract syntax

In [19]:
-- an abstract syntax for the target representation

abstract Target = {
    cat
        Prop; Term;
    fun
        and : Prop -> Prop -> Prop;
        a : Term;
        b : Term;
        p : Term -> Prop;
        q : Term -> Prop;
}

Successfully imported Target.gf

In [20]:
-- we can even make a concrete syntax for it to get nice-looking strings

concrete TargetStr of Target = {
    lincat
        Prop = Str;
        Term = Str;
    lin
        and a b = a ++ "∧" ++ b;
        a = "a";
        b = "b";
        p x = "p" ++ "(" ++ x ++ ")";
        q x = "q" ++ "(" ++ x ++ ")";
}

Successfully imported TargetStr.gf

In [21]:
-- the semantics construction is also defined in an abstract syntax, using the `def` keyword to define functions

abstract Semantics = Grammar, Target ** {
    fun
        transS : S -> Prop;
        transAgent : Agent -> Term;
        transVerb : Verb -> (Term -> Prop);   -- note that the parentheses are optional
    def
        transVerb (and_Verb v1 v2) = \x -> and (transVerb v1 x) (transVerb v2 x);
        transS (and_S s1 s2) = and (transS s1) (transS s2);
        transS (act x v) = (transVerb v) (transAgent x);
        transAgent ahmed = a;
        transAgent berta = b;
        transVerb paint = p;
        transVerb be_quiet = q;        
}

MMT import failed: class info.kwarc.mmt.glf.GfLexerException: string matching regex '[a-zA-Z_][a-zA-Z0-9_]*' expected but '(' found

In [22]:
concrete SemanticsEng of Semantics = GrammarEng ** {
    -- hack as `pt -transfer` is not supported anymore
    lincat Prop = Str;
    lin transS s = s;
}

Successfully imported SemanticsEng.gf

In [25]:
concrete SemanticsStr of Semantics = TargetStr ** { }

Successfully imported SemanticsStr.gf

In [26]:
-- Now we can try out the semantics construction:
parse -lang=Eng -cat=Prop "Ahmed is quiet and paints" | put_tree -compute

and (q a) (p a)

In [27]:
-- And with linearization, we even get a nice looking string:
parse -lang=Eng -cat=Prop "Ahmed is quiet and paints" | put_tree -compute | linearize -lang=Str

q ( a ) ∧ p ( a )